## Older Version

In [1]:
import pandas as pd
import datetime

In [3]:
import win32com.client
outlook = win32com.client.Dispatch("Outlook.Application")
mail = outlook.CreateItem(0x0)
titus_class = mail.UserProperties.Add("TITUSAutomatedClassification",1)
titus_class.Value = 'TLPropertyRoot=ABCDE;Classification=For internal use only; Registered to:My Companies;'
mail.To = 'yash.sethi@mcgill.ca'
mail.Subject = 'Test email'
mail.Body = " This is the first email drafted to test if the automated email works with outlook "
attachment = r"C:\Users\hp\Downloads\AI Upskilling Roadmap by Aditya Sharma.pdf"
mail.Attachments.Add(attachment)
mail.Send()

# Older version with Confidentiality

In [2]:
import win32com.client as win32
from win32com.client import constants

outlook = win32.Dispatch("Outlook.Application")
mail = outlook.CreateItem(0x0)  # olMailItem

# --- TITUS / custom classification (keep your existing line) ---
titus_class = mail.UserProperties.Add("TITUSAutomatedClassification", 1)  # 1 = olText
titus_class.Value = "TLPropertyRoot=ABCDE;Classification=For internal use only; Registered to:My Companies;"

# --- Restricted / Internal controls ---
mail.Sensitivity = constants.olConfidential           # Banner: "Confidential"
try:
    mail.Permission = constants.olDoNotForward        # IRM: prevents forward/print/copy (if enabled)
except Exception:
    # IRM not enabled in profile/tenant – message will still send as Confidential
    pass

# (Optional) add a subject tag many orgs key rules on
subject_tag = "[Restricted]"
# -------------- compose ----------------
mail.To = "yash.sethi@mcgill.ca"
mail.Subject = f"{subject_tag} Test email"
mail.Body = (
    "This is the first email drafted to test if the automated email works with Outlook.\n"
    "Restricted: internal use only."
)
attachment = r"C:\Users\hp\Downloads\AI Upskilling Roadmap by Aditya Sharma.pdf"
mail.Attachments.Add(attachment)

mail.Send()


## New Version

## Have added a Restricted access and reply to the existing thread (latest email)

In [14]:
import datetime as dt
import win32com.client as win32
from win32com.client import constants

def find_last_message_by_subject(ns, subject_hint, folder="Inbox", since_days=30):
    """
    Return the most recent MailItem in 'folder' whose Subject contains subject_hint.
    folder can be "Inbox" or "Sent Items" or any visible subfolder under your default store.
    """
    # Get the folder
    if folder == "Inbox":
        root = ns.GetDefaultFolder(constants.olFolderInbox)
    elif folder == "Sent Items":
        root = ns.GetDefaultFolder(constants.olFolderSentMail)
    else:
        # Try to resolve a custom folder under the default store
        root = ns.GetDefaultFolder(constants.olFolderInbox).Parent.Folders[folder]

    # Restrict to recent items (Outlook expects US mm/dd/yyyy hh:mm AM/PM)
    cutoff = (dt.datetime.now() - dt.timedelta(days=since_days)).strftime("%m/%d/%Y %I:%M %p")
    items = root.Items
    items = items.Restrict(f"[SentOn] >= '{cutoff}' or [ReceivedTime] >= '{cutoff}'")
    items.Sort("[SentOn]", True)

    # Iterate newest → oldest, guard for non-mail items
    hint_lower = subject_hint.lower()
    for it in items:
        try:
            if getattr(it, "Class", None) == constants.olMail and hint_lower in ((it.Subject or "").lower()):
                return it
        except Exception:
            continue
    return None

def reply_in_thread_with_attachment(subject_hint, attachment_path, reply_all=True, folder="Inbox"):
    outlook = win32.Dispatch("Outlook.Application")
    ns = outlook.Session

    anchor = find_last_message_by_subject(ns, subject_hint=subject_hint, folder=folder, since_days=60)
    if not anchor:
        raise RuntimeError(f"No recent message found containing subject: {subject_hint!r} in {folder!r}")

    reply = anchor.ReplyAll() if reply_all else anchor.Reply()

    # Restricted / internal
    reply.Sensitivity = constants.olConfidential
    # try:
    #     reply.Permission = constants.olDoNotForward  # Works only if IRM enabled
    # except Exception:
    #     pass

    # Keep threading (RE: stays). Optional subject tag:
    reply.Subject = f"[Restricted] {reply.Subject}"

    # Short note above the quoted chain
    reply.HTMLBody = "<p>Attaching the latest report as discussed. Please keep internal.</p>" + reply.HTMLBody

    # TITUS property
    titus = reply.UserProperties.Add("TITUSAutomatedClassification", 1)  # 1 = olText
    titus.Value = "TLPropertyRoot=ABCDE;Classification=Restricted;Registered to:My Company;"

    # Attach
    reply.Attachments.Add(attachment_path)

    reply.Send()


In [15]:
reply_in_thread_with_attachment(
    subject_hint="Test email",  # distinctive part of the thread subject
    attachment_path=r"C:\Users\hp\Downloads\AI Upskilling Roadmap by Aditya Sharma.pdf",
    reply_all=True,
    folder="Tests"  # or "Sent Items" if the anchor is there
)
